# DemoWhereIsMySqlAgent.ipynb

<span style="font-size: 14px;">Written by Taiob&nbsp; Ali</span>

<span style="font-size: 14px;">SqlWorldWide.com</span>

## Set up Demo

Setup section of the notebook will create:

-  A resource group
- Two logical SQL server
- Few sample databases
- SQL Elastic Pool
- Clean up code at the end

<mark>Script can take between 40~50 minutes during my test. Mileage will vary in your case</mark>

Credit:

- [https://docs.microsoft.com/en-us/azure/sql-database/sql-database-get-started-powershell](https://docs.microsoft.com/en-us/azure/sql-database/sql-database-get-started-powershell)

<span style="color: rgb(23, 23, 23); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(226, 218, 241);">PowerShell 7 and later is the recommended version of PowerShell for use with Az PowerShell on all platforms.</span>

To check the version of PowerShell running on your machine, run the following command.

If you have an outdated version, upgrade from:  
[https://github.com/PowerShell/PowerShell](https://github.com/PowerShell/PowerShell)

In [ ]:
$PSVersionTable.PSVersion

Install Az Module if you do not have it.  
<span style="font-size: 14px;">https://docs.microsoft.com/en-us/powershell/azure/install-az-ps?view=azps-5.4.0</span>

<span style="font-size: 14px;">If you've disabled module autoloading, manually import the module with `Import-Module -Name Az`. Because of the way the module is structured, this can take a few seconds.</span>

In [ ]:
Import-Module -name Az 

<span style="font-size: 14px;">Setup your Debug Preference</span>

[https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about\_preference\_variables?view=powershell-7.1#debugpreference](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_preference_variables?view=powershell-7.1#debugpreference)  
<span style="font-size: 14px;">This example shows the effect of `$DebugPreference` with the Continue value. The debug message is displayed and the command continues to process.</span>

In [ ]:
$DebugPreference = "Continue"
Write-Debug -Message "Hello, World"

<span style="font-size: 14px;">Sign in to Azure</span>

In [ ]:
Connect-AzAccount

If you need to see the list of your subscription

<span style="color: rgb(106, 153, 85);">$SubscriptionList=Get-AzSubscription</span><span style="color: rgb(106, 153, 85);">$SubscriptionList</span>

Use below code if you have multiple subscription and you want to use a particular one

In [ ]:
Set-AzContext -SubscriptionId '6f8db000-8416-43f0-a2db-cbfb7c945982'

Declare Variables

In [ ]:
$resourceGroupName = "sqlagentdemo"
$primaryLocation = "East US"  
$elasticPoolName = "sqlagentdemo"
$jobServerName = "ugdemojobserver"
$targetServerName = "ugdemotargetserver"
$adminlogin = "taiob"
$password = Get-Content "C:\password.txt"
$ipinfo = Invoke-RestMethod http://ipinfo.io/json 
$startip = $ipinfo.ip
$endip = $ipinfo.ip 
$jobDatabase = "jobdatabase"
$collectionDatabase = "dbawarehouse"
$databaseName1 = "adventureworks"
$databaseName2 = "WideWorldImporters"
$localInstanceName = 'DESKTOP-50O69FS'
$localDatabaseName = 'dbadatabase'
$localTableName = 'databasesize'
$automationAccountName = "ugdemo2"
$credentialName = "sqlservercredentials"

Check if Resource group exists. If exist delete the resource group.

Create a new Resource group

In [ ]:
$resGrpChk = Get-AzResourceGroup `
    -Name $resourceGroupName `
    -ev notPresent `
    -ea 0

if ($resGrpChk) {  
    #Delete resource group
    Remove-AzResourceGroup `
        -Name $resourceGroupName -Confirm   
    Write-Host 'Resource group deleted' `
        -fore white `
        -back green
}

New-AzResourceGroup `
    -Name $resourceGroupName `
    -Location "East US"    
Write-Host 'Resource group created' `
    -fore white `
    -back green

- <span style="font-size: 14px;">Creating job server</span>
- <span style="font-size: 14px;">Configure server firewall rule for job server</span>
- <span style="font-size: 14px;">Create an empty database to hold job metadata</span>
- <span style="font-size: 14px;">Create an empty database to hold job output</span>

In [ ]:
New-AzSqlServer `
    -ResourceGroupName $resourceGroupName `
    -ServerName $jobServerName `
    -Location $primaryLocation `
    -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
    -ArgumentList $adminlogin, $(ConvertTo-SecureString -String $password -AsPlainText -Force))

#Configure a server firewall rule for job server
New-AzSqlServerFirewallRule `
    -ResourceGroupName $resourceGroupName `
    -ServerName $jobServerName `
    -FirewallRuleName "TaiobDesktop" `
    -StartIpAddress $startip `
    -EndIpAddress $endip

# This is done to allow access to Azure Services
New-AzSqlServerFirewallRule `
    -ServerName $jobServerName `
    -ResourceGroupName $resourceGroupName  `
    -AllowAllAzureIPs

New-AzSqlDatabase  -ResourceGroupName $resourceGroupName `
    -ServerName $jobServerName `
    -DatabaseName $jobDatabase `
    -Edition "Standard" `
    -RequestedServiceObjectiveName "S0" `
    -MaxSizeBytes 10737418240 

New-AzSqlDatabase  -ResourceGroupName $resourceGroupName `
    -ServerName $jobServerName `
    -DatabaseName $collectionDatabase `
    -Edition "Standard" `
    -RequestedServiceObjectiveName "S0" `
    -MaxSizeBytes 10737418240 

- <span style="font-size: 14px;">Create target server</span>
- <span style="font-size: 14px;">Configure server firewall rule for target server<br></span>
- <span style="font-size: 14px;">Create a database using adventureworks smaple</span>
- <span style="font-size: 14px;">Create a database for runbook demo&nbsp;</span>

In [ ]:
New-AzSqlServer `
    -ResourceGroupName $resourceGroupName `
    -ServerName $targetServerName `
    -Location $primaryLocation `
    -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
    -ArgumentList $adminlogin, $(ConvertTo-SecureString -String $password -AsPlainText -Force))

#Configure a server firewall rule for target server
New-AzSqlServerFirewallRule `
    -ResourceGroupName $resourceGroupName `
    -ServerName $targetServerName `
    -FirewallRuleName "TaiobDesktop" `
    -StartIpAddress $startip `
    -EndIpAddress $endip

#This is done to allow access to Azure Services 
New-AzSqlServerFirewallRule `
    -ServerName $targetServerName `
    -ResourceGroupName $resourceGroupName  `
    -AllowAllAzureIPs

New-AzSqlDatabase  `
    -ResourceGroupName $resourceGroupName `
    -ServerName $targetServerName `
    -DatabaseName $databasename1 `
    -Edition "Standard" `
    -RequestedServiceObjectiveName "S0" `
    -MaxSizeBytes 10737418240 `
    -SampleName "AdventureWorksLT"

New-AzSqlDatabase  `
    -ResourceGroupName $resourceGroupName `
    -ServerName $targetServerName `
    -DatabaseName testRunBookDB `
    -Edition "Standard" `
    -RequestedServiceObjectiveName "S0" `
    -MaxSizeBytes 10737418240 `

<span style="font-size: 14px;">Create a database using wideworldimporters bacpac file</span>

<span style="font-size: 14px;">https://github.com/Microsoft/sql-server-samples/releases/download/wide-world-importers-v1.0/WideWorldImporters-Standard.bacpac</span>

In [ ]:
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
$url = "http://github.com/Microsoft/sql-server-samples/releases/download/wide-world-importers-v1.0/WideWorldImporters-Standard.bacpac"
$output = "c:\WideWorldImporters-Standard.bacpac"
Invoke-WebRequest -Uri $url -OutFile $output
Set-Location "C:\sqlpackage-win7-x64-en-162.0.52.1\"

.\sqlpackage.exe /a:Import /sf:$output /tsn:"$targetServerName.database.windows.net" `
    /tdn:$databaseName2 /tu:$adminlogin /tp:$password
    

- Create an elastic database pool for a SQL Database
- Create an empty database name test1

In [ ]:
New-AzSqlElasticPool `
    -ResourceGroupName $resourceGroupName `
    -ServerName $targetServerName `
    -ElasticPoolName $elasticPoolName `
    -Edition "Standard" `
    -Dtu 400 `
    -DatabaseDtuMin 10 `
    -DatabaseDtuMax 100

New-AzSqlDatabase  `
    -ResourceGroupName $resourceGroupName `
    -ServerName $targetServerName `
    -DatabaseName "test1" `
    -MaxSizeBytes 10737418240 `
    -ElasticPoolName $elasticPoolName

To use Elastic Jobs, register the feature in your Azure subscription by running the following command (this only needs to be run once in each subscription where you want to use Elastic Jobs)

In [ ]:
Register-AzProviderFeature `
    -FeatureName sqldb-JobAccounts `
    -ProviderNamespace Microsoft.Sql

## Few more settings for Demo to work

- Set up Active Directory Admin for ugdemotargetserver.database.windows.net. This is necessary for function app 'managed service identity' to work with Azure SQL Database
- Create an Azure Fucntion App named SqlAgentDemo with Runtime stack "Powershell Core". Enable system assigned managed identity for the function app
- Run this against testRunBookDB database (You must be connected with an AD Account)
    ```
    CREATE USER SqlAgentDemo FROM EXTERNAL PROVIDER
    GO
    ALTER ROLE db_owner ADD MEMBER SqlAgentDemo
    GO
    
    ```
    
- Create a function with the [code](#function-code) given in this notebook

## Demo starts here

### Demo running script against Azure SQL Database using Linked Server

Use NoteBook  1\_LinkedServer.ipynb

<h3>Demo how to run scripts against all Azure SQL Database in your tenant</h3>

<span style="font-size: 14px;">This script will&nbsp;&nbsp;</span> 

-  Iterate through all resource type SQL Server
- Collect file size of from all Azure SQL Database from all Azure SQL Servers
- Save the result in local On-Premises server

In [23]:
Import-Module -Name SqlServer
# Putting my query in a variable
$databaseQuery = 
"
SELECT 
		GETDATE() AS collectedAT,
		@@SERVERNAME AS serverName, 
		DB_NAME() AS databaseName, 
		LEFT(a.name, 64) AS fileName,
		a.file_id AS fileId,
		a.size AS fileSizeMB,
		CONVERT(DECIMAL(12, 2), ROUND(FILEPROPERTY(a.name,'SpaceUsed')/ 128.000, 2)) AS spaceUsedMB,
		CONVERT(DECIMAL(12, 2), ROUND(( a.size - FILEPROPERTY(a.name,'SpaceUsed'))/ 128.000, 2)) AS freeSpaceMB,
		CONVERT(DECIMAL(12, 2), (CONVERT(DECIMAL(12, 2), ROUND((a.size - FILEPROPERTY(a.name,'SpaceUsed'))/128.000, 2))*100)/ CONVERT(DECIMAL(12, 2), ROUND(a.size / 128.000, 2))) as percentFree,
		a.physical_name AS physicalName 
FROM sys.database_files a
"

$password = ConvertTo-SecureString -String $password -AsPlainText -Force
#$databaseCredentials = Get-Credential -Message "Please provide credentials for $SqlInstance"
$databaseCredentials = New-Object System.Management.Automation.PSCredential($adminlogin, $password) 

#Get all resources type SQL Server, loop through all SQL Server and collect size for each database
$resources = Get-AzResource -ResourceGroupName 'sqlagentdemo' | Where-Object { $_.ResourceType -eq "Microsoft.Sql/servers" } | Select-Object name
foreach ($SqlInstance in $resources) { 
    $SqlInstance = "$($SqlInstance.Name).database.windows.net"
    $databases = Invoke-Sqlcmd -Query "select name from sys.databases" -ServerInstance $SqlInstance `
        -Username $databaseCredentials.GetNetworkCredential().UserName `
        -Password $databaseCredentials.GetNetworkCredential().Password `
        -Database 'master'

    foreach ($databaseName in $databases.name) {
        Write-Host "Query results for database $databaseName.`n"
        Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
            -Username $databaseCredentials.GetNetworkCredential().UserName `
            -Password $databaseCredentials.GetNetworkCredential().Password `
            -Database $databaseName | `
            Write-DbaDbTableData -SqlInstance $localInstanceName -Database $localDatabaseName -Table $localTableName
    }
}

. {
>> Import-Module -Name SqlServer
>> # Putting my query in a variable
>> $databaseQuery = 
>> "
>> SELECT 
>> 		GETDATE() AS collectedAT,
>> 		@@SERVERNAME AS serverName, 
>> 		DB_NAME() AS databaseName, 
>> 		LEFT(a.name, 64) AS fileName,
>> 		a.file_id AS fileId,
>> 		a.size AS fileSizeMB,
>> 		CONVERT(DECIMAL(12, 2), ROUND(FILEPROPERTY(a.name,'SpaceUsed')/ 128.000, 2)) AS spaceUsedMB,
>> 		CONVERT(DECIMAL(12, 2), ROUND(( a.size - FILEPROPERTY(a.name,'SpaceUsed'))/ 128.000, 2)) AS freeSpaceMB,
>> 		CONVERT(DECIMAL(12, 2), (CONVERT(DECIMAL(12, 2), ROUND((a.size - FILEPROPERTY(a.name,'SpaceUsed'))/128.000, 2))*100)/ CONVERT(DECIMAL(12, 2), ROUND(a.size / 128.000, 2))) as percentFree,
>> 		a.physical_name AS physicalName 
>> FROM sys.database_files a
>> "
>> 
>> $password = ConvertTo-SecureString -String $password -AsPlainText -Force
>> #$databaseCredentials = Get-Credential -Message "Please provide credentials for $SqlInstance"
>> $databaseCredentials = New-Object System.Management.

>> #Get all resources type SQL Server, loop through all SQL Server and collect size for each database
>> $resources = Get-AzResource -ResourceGroupName 'sqlagentdemo' | Where-Object { $_.ResourceType -eq "Microsoft.Sql/servers" } | Select-Object name
>> foreach ($SqlInstance in $resources) { 
>>     $SqlInstance = "$($SqlInstance.Name).database.windows.net"
>>     $databases = Invoke-Sqlcmd -Query "select name from sys.databases" -ServerInstance $SqlInstance `
>>         -Username $databaseCredentials.GetNetworkCredential().UserName `
>>         -Password $databaseCredentials.GetNetworkCredential().Password `
>>         -Database 'master'
>> 
>>     foreach ($databaseName in $databases.name) {
>>         Write-Host "Query results for database $databaseName.`n"
>>         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
>>             -Username $databaseCredentials.GetNetworkCredential().UserName `
>>             -Password $databaseCredentials.GetNetworkCredential().Password 

Invoke-Sqlcmd : Login failed for user 'taiob'.
At line:26 char:18
+ ... databases = Invoke-Sqlcmd -Query "select name from sys.databases" -Se ...
+                 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Invoke-Sqlcmd], SqlException
    + FullyQualifiedErrorId : SqlExceptionError,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Invoke-Sqlcmd : Incorrect syntax was encountered while parsing ''.
At line:26 char:18
+ ... databases = Invoke-Sqlcmd -Query "select name from sys.databases" -Se ...
+                 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ParserError: (:) [Invoke-Sqlcmd], BatchParserException
    + FullyQualifiedErrorId : ExecutionFailureException,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Query results for database master.



Invoke-Sqlcmd : Login failed for user 'taiob'.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Invoke-Sqlcmd], SqlException
    + FullyQualifiedErrorId : SqlExceptionError,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Invoke-Sqlcmd : Incorrect syntax was encountered while parsing ''.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ParserError: (:) [Invoke-Sqlcmd], BatchParserException
    + FullyQualifiedErrorId : ExecutionFailureException,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Query results for database adventureworks.



Invoke-Sqlcmd : Login failed for user 'taiob'.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Invoke-Sqlcmd], SqlException
    + FullyQualifiedErrorId : SqlExceptionError,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Invoke-Sqlcmd : Incorrect syntax was encountered while parsing ''.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ParserError: (:) [Invoke-Sqlcmd], BatchParserException
    + FullyQualifiedErrorId : ExecutionFailureException,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Query results for database testRunBookDB.



...

Invoke-Sqlcmd : Login failed for user 'taiob'.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Invoke-Sqlcmd], SqlException
    + FullyQualifiedErrorId : SqlExceptionError,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Invoke-Sqlcmd : Incorrect syntax was encountered while parsing ''.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ParserError: (:) [Invoke-Sqlcmd], BatchParserException
    + FullyQualifiedErrorId : ExecutionFailureException,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Query results for database adventureworks.



Invoke-Sqlcmd : Login failed for user 'taiob'.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Invoke-Sqlcmd], SqlException
    + FullyQualifiedErrorId : SqlExceptionError,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Invoke-Sqlcmd : Incorrect syntax was encountered while parsing ''.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ParserError: (:) [Invoke-Sqlcmd], BatchParserException
    + FullyQualifiedErrorId : ExecutionFailureException,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Query results for database testRunBookDB.



Invoke-Sqlcmd : Login failed for user 'taiob'.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Invoke-Sqlcmd], SqlException
    + FullyQualifiedErrorId : SqlExceptionError,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Invoke-Sqlcmd : Incorrect syntax was encountered while parsing ''.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ParserError: (:) [Invoke-Sqlcmd], BatchParserException
    + FullyQualifiedErrorId : ExecutionFailureException,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Query results for database WideWorldImporters.



Invoke-Sqlcmd : Login failed for user 'taiob'.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Invoke-Sqlcmd], SqlException
    + FullyQualifiedErrorId : SqlExceptionError,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Invoke-Sqlcmd : Incorrect syntax was encountered while parsing ''.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ParserError: (:) [Invoke-Sqlcmd], BatchParserException
    + FullyQualifiedErrorId : ExecutionFailureException,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Query results for database test1.



Invoke-Sqlcmd : Login failed for user 'taiob'.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Invoke-Sqlcmd], SqlException
    + FullyQualifiedErrorId : SqlExceptionError,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 
Invoke-Sqlcmd : Incorrect syntax was encountered while parsing ''.
At line:32 char:9
+         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ParserError: (:) [Invoke-Sqlcmd], BatchParserException
    + FullyQualifiedErrorId : ExecutionFailureException,Microsoft.SqlServer.Management.PowerShell.GetScriptCommand
 


See the result

In [24]:
Set-DbatoolsInsecureConnection
Invoke-DbaQuery `
    -SqlInstance $localInstanceName `
    -Query 'SELECT TOP 20 * FROM DbaDatabase.dbo.databasesize ORDER BY collectedAT DESC ;' | Format-Table -AutoSize



    Module: sql


FullName                 Value Description                 
--------                 ----- -----------                 
sql.connection.trustcert True  Trust SQL Server certificate
sql.connection.encrypt   False Encrypt connection to server



collectedAT          serverName         databaseName   fileName fileId fileSizeMB spaceUsedMB freeSpaceMB percentFree p
                                                                                                                      h
                                                                                                                      y
                                                                                                                      s
                                                                                                                      i
                                                                                                                      c
                 

### Demo  Windows Task Scheduler

Run task: 'CollectAzureDatabaseSize' from Windows Task Scheduler

If we want to do  the same and we do not have any on-premises SQL Server; we can schedule the same using any windows host. Similary we can do from an IaaS VM.

Following xml will create the task. You will need to change few values.

\<?xml version="1.0" encoding="UTF-16"?\>

\<Task version="1.4" xmlns="http://schemas.microsoft.com/windows/2004/02/mit/task"\>

  \<RegistrationInfo\>

    \<Date\>2022-10-22T20:18:38.8419638\</Date\>

    \<Author\>DESKTOP-50O69FS\\taiob\</Author\>

    \<URI\>\\CollectAzureDatabaseSize\</URI\>

  \</RegistrationInfo\>

  \<Triggers /\>

  \<Principals\>

    \<Principal id="Author"\>

      \<UserId\>S-1-5-21-1134363470-949122783-3418734209-1001\</UserId\>

      \<LogonType\>InteractiveToken\</LogonType\>

      \<RunLevel\>HighestAvailable\</RunLevel\>

    \</Principal\>

  \</Principals\>

  \<Settings\>

    \<MultipleInstancesPolicy\>IgnoreNew\</MultipleInstancesPolicy\>

    \<DisallowStartIfOnBatteries\>false\</DisallowStartIfOnBatteries\>

    \<StopIfGoingOnBatteries\>true\</StopIfGoingOnBatteries\>

    \<AllowHardTerminate\>true\</AllowHardTerminate\>

    \<StartWhenAvailable\>false\</StartWhenAvailable\>

    \<RunOnlyIfNetworkAvailable\>false\</RunOnlyIfNetworkAvailable\>

    \<IdleSettings\>

      \<StopOnIdleEnd\>true\</StopOnIdleEnd\>

      \<RestartOnIdle\>false\</RestartOnIdle\>

    \</IdleSettings\>

    \<AllowStartOnDemand\>true\</AllowStartOnDemand\>

    \<Enabled\>true\</Enabled\>

    \<Hidden\>false\</Hidden\>

    \<RunOnlyIfIdle\>false\</RunOnlyIfIdle\>

    \<DisallowStartOnRemoteAppSession\>false\</DisallowStartOnRemoteAppSession\>

    \<UseUnifiedSchedulingEngine\>true\</UseUnifiedSchedulingEngine\>

    \<WakeToRun\>false\</WakeToRun\>

    \<ExecutionTimeLimit\>PT72H\</ExecutionTimeLimit\>

    \<Priority\>7\</Priority\>

  \</Settings\>

  \<Actions Context="Author"\>

    \<Exec\>

      \<Command\>"C:\\Program Files\\PowerShell\\7\\pwsh.exe"\</Command\>

      \<Arguments\>-File "C:\\Presentation\\Azure SQL Database - Where is my  SQL Agent\\2\_CollectDatabaseSize.ps1"\</Arguments\>

    \</Exec\>

  \</Actions\>

\</Task\>

### Demo  Azure Automation

- Create an Automation account
- Create an empty runbook type PowerShellWorkflow

In [ ]:
$automationAccountName = "ugdemo2"
$credentialName = "sqlservercredentials"
$password = Get-Content "C:\password.txt"
$passwordSecure =ConvertTo-SecureString $password -AsPlainText -Force
$credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $adminlogin, $passwordSecure

#Creating an automation account
New-AzAutomationAccount `
    -ResourceGroupName $resourceGroupName `
    -Name $automationAccountName `
    -Location $primaryLocation

#Creating credential
New-AzAutomationCredential `
    -AutomationAccountName $automationAccountName `
    -Name $credentialName `
    -Value $credential `
    -ResourceGroupName $resourceGroupName

#Creating a blank runbook
New-AzAutomationRunbook `
    -ResourceGroupName $resourceGroupName `
    -AutomationAccountName $automationAccountName `
    -Name 'Update-SQLIndexRunbook' `
    -Type 'PowerShellWorkflow' `
    -LogProgress $true `
    -LogVerbose $true

Go to portal and demo runbook

Replace the generic code with PowerShell code from below cell

**Demo Runbook** using 3\_TestRunBook.sql

In [ ]:
<#
.SYNOPSIS 
    Indexes tables in a database if they have a high fragmentation

.DESCRIPTION
    This runbook indexes all of the tables in a given database if the fragmentation is
    above a certain percentage. 
    It highlights how to break up calls into smaller chunks, 
    in this case each table in a database, and use checkpoints. 
    This allows the runbook job to resume for the next chunk of work even if the 
    fairshare feature of Azure Automation puts the job back into the queue every 30 minutes

.PARAMETER SqlServer
    Name of the SqlServer

.PARAMETER Database
    Name of the database
    
.PARAMETER SQLCredentialName
    Name of the Automation PowerShell credential setting from the Automation asset store. 
    This setting stores the username and password for the SQL Azure server

.PARAMETER FragPercentage
    Optional parameter for specifying over what percentage fragmentation to index database
    Default is 20 percent
 
 .PARAMETER RebuildOffline
    Optional parameter to rebuild indexes offline if online fails 
    Default is false
    
 .PARAMETER Table
    Optional parameter for specifying a specific table to index
    Default is all tables
    
.PARAMETER SqlServerPort
    Optional parameter for specifying the SQL port 
    Default is 1433
    
.EXAMPLE
    Update-SQLIndexRunbook -SqlServer "server.database.windows.net" -Database "Finance" -SQLCredentialName "FinanceCredentials"

.EXAMPLE
    Update-SQLIndexRunbook -SqlServer "server.database.windows.net" -Database "Finance" -SQLCredentialName "FinanceCredentials" -FragPercentage 30

.EXAMPLE
    Update-SQLIndexRunbook -SqlServer "server.database.windows.net" -Database "Finance" -SQLCredentialName "FinanceCredentials" -Table "Customers" -RebuildOffline $True

.NOTES
    AUTHOR: System Center Automation Team
    LASTEDIT: Oct 8th, 2014 
    Modified By: Taiob Ali
    Last Modified: March 2019
#>
workflow Update-SQLIndexRunbook
{
    param(
        [parameter(Mandatory=$True)]
        [string] $SqlServer = "ugdemotargetserver.database.windows.net",
    
        [parameter(Mandatory=$True)]
        [string] $Database = "testrunbookdb",
    
        [parameter(Mandatory=$True)]
        [string] $SQLCredentialName = "sqlservercredentials",
            
        [parameter(Mandatory=$False)]
        [int] $FragPercentage = 20,

        [parameter(Mandatory=$False)]
        [int] $SqlServerPort = 1433,
        
        [parameter(Mandatory=$False)]
        [boolean] $RebuildOffline = $False,

        [parameter(Mandatory=$False)]
        [string] $Table = "testRebuild"
                  
    )

    # Get the stored username and password from the Automation credential
    $SqlCredential = Get-AutomationPSCredential -Name $SQLCredentialName
    if ($SqlCredential -eq $null)
    {
        throw "Could not retrieve '$SQLCredentialName' credential asset. Check that you created this first in the Automation service."
    }
    
    $SqlUsername = $SqlCredential.UserName 
    $SqlPass = $SqlCredential.GetNetworkCredential().Password
    
    $TableNames = Inlinescript {
      
        # Define the connection to the SQL Database
        $Conn = New-Object System.Data.SqlClient.SqlConnection("Server=tcp:$using:SqlServer,$using:SqlServerPort;Database=$using:Database;User ID=$using:SqlUsername;Password=$using:SqlPass;Trusted_Connection=False;Encrypt=True;Connection Timeout=30;")
         
        # Open the SQL connection
        $Conn.Open()
        
        # SQL command to find tables and their average fragmentation
        $SQLCommandString = @"
        SELECT a.object_id, avg_fragmentation_in_percent
        FROM sys.dm_db_index_physical_stats (
               DB_ID(N'$Database')
             , OBJECT_ID(0)
             , NULL
             , NULL
             , NULL) AS a
        JOIN sys.indexes AS b 
        ON a.object_id = b.object_id AND a.index_id = b.index_id;
"@
        # Return the tables with their corresponding average fragmentation
        $Cmd=new-object system.Data.SqlClient.SqlCommand($SQLCommandString, $Conn)
        $Cmd.CommandTimeout=120
        
        # Execute the SQL command
        $FragmentedTable=New-Object system.Data.DataSet
        $Da=New-Object system.Data.SqlClient.SqlDataAdapter($Cmd)
        [void]$Da.fill($FragmentedTable)

 
        # Get the list of tables with their object ids
        $SQLCommandString = @"
        SELECT  t.name AS TableName, t.OBJECT_ID FROM sys.tables t
"@

        $Cmd=new-object system.Data.SqlClient.SqlCommand($SQLCommandString, $Conn)
        $Cmd.CommandTimeout=120

        # Execute the SQL command
        $TableSchema =New-Object system.Data.DataSet
        $Da=New-Object system.Data.SqlClient.SqlDataAdapter($Cmd)
        [void]$Da.fill($TableSchema)


        # Return the table names that have high fragmentation
        ForEach ($FragTable in $FragmentedTable.Tables[0])
        {
            Write-Verbose ("Table Object ID:" + $FragTable.Item("object_id"))
            Write-Verbose ("Fragmentation:" + $FragTable.Item("avg_fragmentation_in_percent"))
            
            If ($FragTable.avg_fragmentation_in_percent -ge $Using:FragPercentage)
            {
                # Table is fragmented. Return this table for indexing by finding its name
                ForEach($Id in $TableSchema.Tables[0])
                {
                    if ($Id.OBJECT_ID -eq $FragTable.object_id.ToString())
                     {
                        # Found the table name for this table object id. Return it
                        Write-Verbose ("Found a table to index! : " +  $Id.Item("TableName"))
                        $Id.TableName
                    }
                }
            }
        }

        $Conn.Close()
    }

    # If a specific table was specified, then find this table if it needs to indexed, otherwise
    # set the TableNames to $null since we shouldn't process any other tables.
    If ($Table)
    {
        Write-Verbose ("Single Table specified: $Table")
        If ($TableNames -contains $Table)
        {
            $TableNames = $Table
        }
        Else
        {
            # Remove other tables since only a specific table was specified.
            Write-Verbose ("Table not found: $Table")
            $TableNames = $Null
        }
    }

    # Interate through tables with high fragmentation and rebuild indexes
    ForEach ($TableName in $TableNames)
    {
      Write-Verbose "Creating checkpoint"
      Checkpoint-Workflow
      Write-Verbose "Indexing Table $TableName..."
      
      InlineScript {
          
        $SQLCommandString = @"
        EXEC('ALTER INDEX ALL ON $Using:TableName REBUILD with (ONLINE=ON)')
"@

        # Define the connection to the SQL Database
        $Conn = New-Object System.Data.SqlClient.SqlConnection("Server=tcp:$using:SqlServer,$using:SqlServerPort;Database=$using:Database;User ID=$using:SqlUsername;Password=$using:SqlPass;Trusted_Connection=False;Encrypt=True;Connection Timeout=30;")
        
        # Open the SQL connection
        $Conn.Open()

        # Define the SQL command to run. In this case we are getting the number of rows in the table
        $Cmd=new-object system.Data.SqlClient.SqlCommand($SQLCommandString, $Conn)
        # Set the Timeout to be less than 30 minutes since the job will get queued if > 30
        # Setting to 25 minutes to be safe.
        $Cmd.CommandTimeout=1500

        # Execute the SQL command
        Try 
        {
            $Ds=New-Object system.Data.DataSet
            $Da=New-Object system.Data.SqlClient.SqlDataAdapter($Cmd)
            [void]$Da.fill($Ds)
        }
        Catch
        {
            if (($_.Exception -match "offline") -and ($Using:RebuildOffline) )
            {
                Write-Verbose ("Building table $Using:TableName offline")
                $SQLCommandString = @"
                EXEC('ALTER INDEX ALL ON $Using:TableName REBUILD')
"@              

                # Define the SQL command to run. 
                $Cmd=new-object system.Data.SqlClient.SqlCommand($SQLCommandString, $Conn)
                # Set the Timeout to be less than 30 minutes since the job will get queued if > 30
                # Setting to 25 minutes to be safe.
                $Cmd.CommandTimeout=1500

                # Execute the SQL command
                $Ds=New-Object system.Data.DataSet
                $Da=New-Object system.Data.SqlClient.SqlDataAdapter($Cmd)
                [void]$Da.fill($Ds)
            }
            Else
            {
                # Will catch the exception here so other tables can be processed.
                Write-Error "Table $Using:TableName could not be indexed. Investigate indexing each index instead of the complete table $_"
             }
        }
        # Close the SQL connection
        $Conn.Close()
      }  
    }

    Write-Verbose "Finished Indexing"
}

### Demo Elastic Job Agent

<span style="font-size: 14px;">This script will create an</span> elastic job agent       

<span style="font-size: 14px;">Credit:</span>

- <u>https://learn.microsoft.com/en-us/azure/azure-sql/database/elastic-jobs-powershell-create?view=azuresql#create-the-elastic-job-agent</u>
- <u>https://learn.microsoft.com/en-us/powershell/module/az.sql/new-azsqlelasticjobagent?view=azps-9.0.1&viewFallbackFrom=azps-5.4.0</u>

Use 4\_ElasticJobAgent.sql to demo Elastic Job Agent

In [ ]:
New-AzSqlElasticJobAgent `
    -ResourceGroupName 'sqlagentdemo' `
    -ServerName 'ugdemojobserver' `
    -DatabaseName 'jobdatabase' `
    -Name 'agentdemo'

### Demo Azure Function

<a id=function-code />

Replace the built in code for Azure function  

<span style="font-size: 14px;">Please refer to my blog post about setting up a function app</span>

<span style="font-size: 14px;"><a href="http://sqlworldwide.com/how-to-use-managed-identity-with-azure-function-app/" data-href="http://sqlworldwide.com/how-to-use-managed-identity-with-azure-function-app/" title="http://sqlworldwide.com/how-to-use-managed-identity-with-azure-function-app/">http://sqlworldwide.com/how-to-use-managed-identity-with-azure-function-app/</a></span>

**Demo Azure Function** using 3\_TestRunBook.sql

In [ ]:
# Input bindings are passed in via param block.
param($Timer)

# Get the current universal time in the default string format
$currentUTCtime = (Get-Date).ToUniversalTime()

# The 'IsPastDue' porperty is 'true' when the current function invocation is later than scheduled.
if ($Timer.IsPastDue) {
    Write-Host "PowerShell timer is running late!"
}
<#
 This function app is using 'Managed Service Identity' to connect to the Azure SQL Database
 In the interest of time I will not show the set up
 Please see details from my blog post:
 http://sqlworldwide.com/how-to-use-managed-identity-with-azure-function-app/
 
 Used help from following resources in setting up 'Managed Service Identity'
 https://docs.microsoft.com/en-us/azure/app-service/app-service-web-tutorial-connect-msi
 https://www.azurecorner.com/using-managed-service-identity-in-azure-functions-to-access-azure-sql-database/
 https://docs.microsoft.com/en-us/azure/app-service/overview-managed-identity?tabs=powershell
#>

$resourceURI = "https://database.windows.net/"
$tokenAuthURI = $env:MSI_ENDPOINT + "?resource=$resourceURI&api-version=2017-09-01"
$tokenResponse = Invoke-RestMethod -Method Get -Headers @{"Secret" = "$env:MSI_SECRET" } -Uri $tokenAuthURI
$accessToken = $tokenResponse.access_token

$SqlConnection = New-Object System.Data.SqlClient.SqlConnection
$SqlConnection.ConnectionString = "Data Source =ugdemotargetserver.database.windows.net ; Initial Catalog = testRunBookDB"
$SqlConnection.AccessToken = $AccessToken
$SqlConnection.Open()

$SqlCmd = New-Object System.Data.SqlClient.SqlCommand
$SqlCmd.CommandText = "ALTER INDEX ALL ON testRebuild REBUILD;"
$SqlCmd.Connection = $SqlConnection
$SqlAdapter = New-Object System.Data.SqlClient.SqlDataAdapter
$SqlAdapter.SelectCommand = $SqlCmd
$DataSet = New-Object System.Data.DataSet
$SqlAdapter.Fill($DataSet) 

### Demo Azure Data Factory

using 3\_TestRunBook.sql

<span style="font-size: 14px;">Clean up by removing resource group name</span>

In [ ]:
Remove-AzResourceGroup -ResourceGroupName $resourceGroupName -Force